In [1]:
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM5', 'COM4']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(3)
TTC.set_alice_transmission_channel(4)
# set bob channels
TTC.set_bob_reflection_channel(2)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()



In [5]:
_ = TTC.performDelayAdjustment(integration_time=5)# manual_delays=[0, 240, -582, 123])

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   308 ps 	/    92.4 mm
Bob_T   :  -555 ps 	/  -166.5 mm
Bob_R   :   121 ps 	/    36.3 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :    49 ps 	/    14.7 mm
Bob_T   :   -66 ps 	/   -19.8 mm
Bob_R   :   -13 ps 	/    -3.9 mm


In [46]:
KMC.rotate_simulataneously(45, 0)

In [24]:

KMC.rotate_simulataneously(0, 0)

In [23]:
KMC.rotate_simulataneously(45, 0)

In [17]:
n_values = 150
trace_width = 3
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

In [11]:
KMC.rotate_simulataneously(0, 0, wait_for_completion=False)

In [12]:

KMC.rotate_simulataneously(45, 0, wait_for_completion=False)

In [9]:

SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0, detector_efficiencies=[0,1,1,1], debug=False)
CHSH_angles = SimulatorPhiPlus.CHSH_angles_for_filters

In [10]:
SimulatorPhiPlus.outcome_probabilities(0, 25)

array([[0.        ],
       [0.01721542],
       [0.01721542],
       [0.96556915]])

In [11]:
TTC.measureS(CHSH_angles=CHSH_angles, integration_time_per_basis_setting_SI=2, TTSimulator=SimulatorPhiPlus, coincidence_window_SI=1.0e-9, debug=True)

AttributeError: 'NoneType' object has no attribute 'home'

In [ ]:
# TODO
# coincidence channel handling and destruction
#TTC.coincidences_vchannels.getChannels()

In [6]:
# TODO This kills the coincidence channels in the visualisation. Needs to be worked on
idealAngles = [0, -22.5, -11.25, 11.25] # for triplett |HH> - |VV> initial state
johannes_ideal_angles = [0, 45/2, -22.5/2, 22.5/2]
johannes_classical_angles = [0, 45/2, -90/2, -45/2]

In [7]:
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=2, TTSimulator=None, coincidence_window_SI=1.0e-9, debug=True)


corr[a,b] = 0.81761
	N[|T,T>]=   259	(0.136)
	N[|T,R>]=   137	(0.072)
	N[|R,T>]=    37	(0.019)
	N[|R,R>]=  1475	(0.773)

corr[a,B] = 0.58093
	N[|T,T>]=   201	(0.111)
	N[|T,R>]=   305	(0.169)
	N[|R,T>]=    73	(0.040)
	N[|R,R>]=  1225	(0.679)

corr[A,b] = 0.45262
	N[|T,T>]=   202	(0.119)
	N[|T,R>]=   390	(0.230)
	N[|R,T>]=    75	(0.044)
	N[|R,R>]=  1032	(0.607)

corr[A,B] = -0.55895
	N[|T,T>]=    58	(0.037)
	N[|T,R>]=  1046	(0.667)
	N[|R,T>]=   177	(0.113)
	N[|R,R>]=   288	(0.184)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.4101152627559705


In [22]:
TTC.measure_S_with_two_ports(CHSH_angles=idealAngles, debug=True)

SPCM Pairs: ['TT', 'TR', 'RT', 'RR']

corr[a,b] = [ 0.811 -0.827 -0.798  0.786]
	N[a , b ]=[1106  173  129 1617]
	N[a , b ]=[ 119 1744 1015  213]
	N[a , b ]=[  84 1411  777  112]
	N[a , b ]=[ 842  126   72 1102]

corr[a,B] = [ 0.522 -0.511 -0.503  0.491]
	N[a , B ]=[ 903  463  266 1375]
	N[a , B ]=[ 276 1482  861  423]
	N[a , B ]=[ 234 1156  604  357]
	N[a , B ]=[722 390 218 907]

corr[A,b] = [ 0.384 -0.442 -0.358  0.402]
	N[A , b ]=[ 783  398  252 1083]
	N[A , b ]=[ 273 1323  732  320]
	N[A , b ]=[ 402 1156  644  553]
	N[A , b ]=[734 562 399 966]

corr[A,B] = [-0.7    0.744  0.641 -0.696]
	N[A , B ]=[ 207 1432  729  322]
	N[A , B ]=[ 909  247  210 1140]
	N[A , B ]=[ 993  195  143 1389]
	N[A , B ]=[ 129 1584  885  131]

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = TT, TR, RT, RR : [2.41735482 2.52424128 2.30037829 2.37563537]


In [ ]:
SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0.00, debug=False)
SimulatorPhiMinus = TT_Simulator(two_particle_states['phi_minus'], initial_state_noise=0.00, debug=False)
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=SimulatorPhiPlus, coincidence_window_SI=0.5e-9)

NameError: name 'TT_Simulator' is not defined

# OSC Server Setup


In [7]:
my_ip = '192.168.0.2'
my_port = 7401 
clemens_port = 7401
visual_port = 7403

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=clemens_port, name='Clemens')
Visuals = OSCTarget(ip=my_ip, port=visual_port, name='Visuals')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


In [8]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Clemens, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Clemens


In [9]:
OSC.send_visuals(target=Visuals, measurement_results="hi")

Measurement hi sent to Visuals


In [10]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    
    time_of_last_visual = time.time()
    while performance_running:
        theta_a, theta_b = random.choice(angle_pair_dict[OSC.currently_selected_state])
        
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        
        # check if setup is measuring Quantum or Classical angles for conversion to clemens format        
        if OSC.currently_selected_state[0] == 'Q':
            check_angles = Q_angles
        elif OSC.currently_selected_state[0] == 'C':
            check_angles = C_angles

        resultClemensFormat = [1 if theta_a == check_angles[0] else 2, 1 if theta_b == check_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

        #if time.time() - time_of_last_visual > 7:
        #    OSC.send_visuals(Visuals, resultClemensFormat)
        #    time_of_last_visual = time.time()


def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [ ]:
Myself.client.send_message("/bruQner/set/state", 'Q_all')

In [ ]:
stop_performance()

performance stopped...


In [12]:
start_performance()

Exception in thread Thread-21 (performance_cycle):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\ghana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ghana\AppData\Local\Temp\ipykernel_18348\1005914504.py", line 29, in performance_cycle
  File "\\wsl.localhost\Ubuntu\home\benjaminorthner\projects\bruQner\calibration_and_testing\..\src\time_tagger\time_tagger_controller.py", line 550, in get_single_measurement
    eventsByChannel = self.collect_stream_data(in